In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from math import radians

In [2]:
pref = 'recreativo'

In [3]:
df1 = pd.read_csv('reto_precios.csv')
df2 = pd.read_csv('recreativos.csv', encoding='latin-1')

C:\Users\alber\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
df2.columns

Index(['id', 'clee', 'nom_estab', 'raz_social', 'codigo_act', 'nombre_act',
       'per_ocu', 'tipo_vial', 'nom_vial', 'tipo_v_e_1', 'nom_v_e_1',
       'tipo_v_e_2', 'nom_v_e_2', 'tipo_v_e_3', 'nom_v_e_3', 'numero_ext',
       'letra_ext', 'edificio', 'edificio_e', 'numero_int', 'letra_int',
       'tipo_asent', 'nomb_asent', 'tipoCenCom', 'nom_CenCom', 'num_local',
       'cod_postal', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'cve_loc',
       'localidad', 'ageb', 'manzana', 'telefono', 'correoelec', 'www',
       'tipoUniEco', 'latitud', 'longitud', 'fecha_alta'],
      dtype='object')

In [5]:
df1 = df1[['id','lat','lon']]
# Cambiar los nombres de las variables en df1 agregando sufijo 1
df1 = df1.rename(columns={ 'lat': 'lat1', 'lon': 'lon1'})
# Cambiar los nombres de las variables en df2 agregando sufijo 2
df2 = df2.rename(columns={'id': 'id2', 'latitud': 'lat2', 'longitud': 'lon2'})


In [6]:
# Definir la tolerancia espacial (en kilómetros)
tolerancia = 0.5

# Convertir las latitudes y longitudes a radianes
df1['lat_rad1'] = df1['lat1'].apply(radians)
df1['lon_rad1'] = df1['lon1'].apply(radians)
df2['lat_rad2'] = df2['lat2'].apply(radians)
df2['lon_rad2'] = df2['lon2'].apply(radians)

In [7]:
# Calcular la distancia haversine entre las coordenadas
distances = haversine_distances(df1[['lat_rad1', 'lon_rad1']], df2[['lat_rad2', 'lon_rad2']]) * 6371000/1000  # Convertir a kilómetros

# Encontrar las coincidencias dentro de la tolerancia
matches = distances <= tolerancia

In [8]:
matching_indices = np.where(matches)

# Crear una lista para almacenar los DataFrames combinados
combined_dfs = []

# Iterar sobre los índices y columnas de coincidencias
for idx, col in zip(*matching_indices):
    # Obtener los registros coincidentes de df1 y df2
    matching_record_df1 = df1.loc[idx]
    matching_record_df2 = df2.loc[col]
    
    # Combinar los registros en un DataFrame
    combined_record = pd.concat([matching_record_df1, matching_record_df2])
    combined_dfs.append(combined_record)

# Concatenar todos los DataFrames combinados en uno solo
final_combined_df = pd.concat(combined_dfs, axis=1).T

In [9]:
final_combined_df.head(3)

,id,lat1,lon1,lat_rad1,lon_rad1,id2,clee,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipo_vial,nom_vial,tipo_v_e_1,nom_v_e_1,tipo_v_e_2,nom_v_e_2,tipo_v_e_3,nom_v_e_3,numero_ext,letra_ext,edificio,edificio_e,numero_int,letra_int,tipo_asent,nomb_asent,tipoCenCom,nom_CenCom,num_local,cod_postal,cve_ent,entidad,cve_mun,municipio,cve_loc,localidad,ageb,manzana,telefono,correoelec,www,tipoUniEco,lat2,lon2,fecha_alta,lat_rad2,lon_rad2
0,1.0,19.421317,-99.164182,0.338966,-1.730741,894816,09015713943001022000000000U1,AKSYSTEMS,NaN,713943,Centros de acondicionamiento físico del sector...,6 a 10 personas,EJE VIAL,2 PONIENTE (FLORENCIA),CALLE,LIVERPOOL,CALLE,LONDRES,CALLE,VARSOVIA,51.0,NaN,EDIFICIO SIN NOMBRE,PISO 1,NaN,NaN,COLONIA,JUAREZ,NaN,NaN,NaN,6600.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,095A,18,5555338599.0,NaN,NaN,Fijo,19.424195,-99.166338,2014-12,0.339016,-1.730779
1,1.0,19.421317,-99.164182,0.338966,-1.730741,7732163,09015711111000253000000000U9,ARTESTUDIO,NaN,711111,Compañías de teatro del sector privado,11 a 30 personas,AVENIDA,INSURGENTES SUR,CALLE,DURANGO,CALLE,COLIMA,CALLE,TONALÁ,214.0,NaN,NaN,NaN,0.0,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,1002,16,NaN,INFO@ARTESTUDIO.COM,WWW.ARTESTUDIO.COM,Fijo,19.420026,-99.163925,2019-11,0.338943,-1.730737
2,1.0,19.421317,-99.164182,0.338966,-1.730741,8842069,09015711510000421000000000U5,"ARTISTAS, ESCRITORES Y TÉCNICOS INDEPENDIENTES",NaN,711510,"Artistas, escritores y técnicos independientes",0 a 5 personas,CALLE,DURANGO,CALLE,JALAPA,CALLE,TONALA,CALLE,COLIMA,118.0,NaN,NaN,NaN,NaN,NaN,COLONIA,ROMA NORTE,NaN,NaN,NaN,6700.0,9,CIUDAD DE MÉXICO,15,Cuauhtémoc,1,Cuauhtémoc,1002,4,5551054618.0,NaN,WWW.JAVIERAREAN.COM,Fijo,19.420617,-99.162173,2019-11,0.338954,-1.730706


In [10]:
len(final_combined_df)

18909

In [11]:
final_combined_df['numero_ext'].value_counts()

11.0     741
99.0     713
0.0      698
133.0    502
155.0    492
        ... 
555.0      1
243.0      1
92.0       1
254.0      1
78.0       1
Name: numero_ext, Length: 200, dtype: int64

In [12]:
final_combined_df['manzana'].value_counts()

3     1404
6     1401
16    1278
10    1166
13    1134
19    1133
18    1132
2     1112
21    1043
25     971
14     929
1      843
9      638
23     615
4      601
20     588
7      483
12     448
11     445
28     329
24     292
5      259
8      194
27     126
26      85
15      84
22      46
46      41
17      35
29      27
33      12
74       5
44       3
59       3
48       2
32       1
34       1
Name: manzana, dtype: int64

In [13]:
final_combined_df['per_ocu'].value_counts()

0 a 5 personas        10290
11 a 30 personas       3594
6 a 10 personas        3529
101 a 250 personas      561
31 a 50 personas        484
51 a 100 personas       413
251 y más personas       38
Name: per_ocu, dtype: int64

In [14]:
personal_ocupado = {'0 a 5 personas' : 1,
'6 a 10 personas' : 2,
'11 a 30 personas' : 3,
'31 a 50 personas' : 4,
'51 a 100 personas' : 5,
'101 a 250 personas' : 6,
'251 y más personas' : 7}

final_combined_df['per_ocu'] = final_combined_df['per_ocu'].replace(personal_ocupado)
final_combined_df['per_ocu'].value_counts()

1    10290
3     3594
2     3529
6      561
4      484
5      413
7       38
Name: per_ocu, dtype: int64

In [15]:
#per_ocu
#numero_ext
#manzana

In [16]:
final_combined_df['dif_lat_rad'] =final_combined_df['lat_rad1']-final_combined_df['lat_rad2']
final_combined_df['dif_lon_rad'] =final_combined_df['lon_rad1']-final_combined_df['lon_rad2']

In [17]:
(final_combined_df['lat_rad1']-final_combined_df['lat_rad2'])

0        -0.00005
1        0.000023
2        0.000012
3         0.00003
4        0.000039
           ...   
18904   -0.000052
18905   -0.000032
18906   -0.000066
18907   -0.000059
18908   -0.000059
Length: 18909, dtype: object

In [18]:
final_combined_df.entidad.value_counts()

CIUDAD DE MÉXICO    18691
NUEVO LEÓN            217
BAJA CALIFORNIA         1
Name: entidad, dtype: int64

In [19]:
final_combined_df[final_combined_df.entidad.str.contains('NUEVO LEÓN')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad
1012,54.0,0.00006,-0.000021
1013,54.0,0.000048,-0.000006
1014,54.0,-0.00006,0.000035
1015,54.0,0.000026,0.000008
1016,54.0,-0.000001,0.000054
...,...,...,...
18312,952.0,0.000025,-0.000073
18313,952.0,-0.000001,-0.000028
18314,952.0,-0.000012,0.000058
18315,952.0,-0.00001,0.000047


In [20]:
final_combined_df[final_combined_df.entidad.str.contains('BAJA CALIFORNIA')][['id','dif_lat_rad','dif_lon_rad']]

,id,dif_lat_rad,dif_lon_rad
11381,583.0,0.000063,0.00004


In [21]:
summary = final_combined_df[['id','per_ocu','numero_ext','manzana']]
summary.head()

,id,per_ocu,numero_ext,manzana
0,1.0,2,51.0,18
1,1.0,3,214.0,16
2,1.0,1,118.0,4
3,1.0,1,225.0,5
4,1.0,1,224.0,16


In [22]:
# Definir las columnas para la agrupación y agregación
aggregation_columns = ['per_ocu', 'numero_ext', 'manzana']  # Agrega aquí las columnas que quieras sumar

# Realizar la agrupación y las agregaciones
grouped_df = summary.groupby('id')[aggregation_columns].agg({'sum'}).reset_index()
grouped_df2 = summary.groupby('id').agg(total=('id', 'count')).reset_index()
grouped_df2.columns = ['id'] + [f'{pref}_total']



# Renombrar las columnas resultantes
grouped_df.columns = ['id'] + [f'{pref}_suma_{col}' for col in aggregation_columns]

# Mostrar el DataFrame resultante
centros = grouped_df2.merge(grouped_df)

In [23]:
centros.head()

,id,recreativo_total,recreativo_suma_per_ocu,recreativo_suma_numero_ext,recreativo_suma_manzana
0,1.0,25,46,3583.0,272
1,2.0,26,46,3382.0,308
2,3.0,10,22,1822.0,98
3,4.0,8,17,1725.0,48
4,5.0,18,35,1866.0,353


In [24]:
centros.to_csv('centros_'+pref+'s'+'.csv')